In [2]:
import pandas as pd
import os

# --- Configuration ---
file_name = "250331 Leng Survey. Full download. Editable.xlsx"
file_path = file_name # Or provide the full path

# --- Reading all sheets ---
try:
    all_sheets_dict = pd.read_excel(file_path, sheet_name=None)
    print("Successfully read all sheets.")
    print(f"Sheet names found: {list(all_sheets_dict.keys())}")

    # --- Accessing the DataFrames ---
    # You now have DataFrames for each sheet in the dictionary:

    df_all_responses = all_sheets_dict['All responses']
    df_pa_survey = all_sheets_dict['PA Survey']
    df_aa_survey = all_sheets_dict['AA Survey']

    # --- Example: Print shapes or heads ---
    print("\n--- DataFrame Details ---")
    print(f"Shape of 'All responses': {df_all_responses.shape}")
    # print("Head of 'All responses':\n", df_all_responses.head()) # Uncomment to view

    print(f"\nShape of 'PA Survey': {df_pa_survey.shape}")
    # print("Head of 'PA Survey':\n", df_pa_survey.head()) # Uncomment to view

    print(f"\nShape of 'AA Survey': {df_aa_survey.shape}")
    # print("Head of 'AA Survey':\n", df_aa_survey.head()) # Uncomment to view

    # Or iterate through them:
    print("\n--- Iterating ---")
    for sheet_name, df in all_sheets_dict.items():
        print(f"Processing sheet: '{sheet_name}' with shape {df.shape}")
        # Add your analysis for each df here

except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

Successfully read all sheets.
Sheet names found: ['All responses', 'PA Survey', 'AA Survey']

--- DataFrame Details ---
Shape of 'All responses': (8558, 137)

Shape of 'PA Survey': (6864, 137)

Shape of 'AA Survey': (1694, 137)

--- Iterating ---
Processing sheet: 'All responses' with shape (8558, 137)
Processing sheet: 'PA Survey' with shape (6864, 137)
Processing sheet: 'AA Survey' with shape (1694, 137)


In [3]:
import pandas as pd
import numpy as np

# --- ASSUMPTION ---
# This code assumes the 'all_sheets_dict' dictionary containing your DataFrames
# already exists in your environment from the previous steps.
# It specifically expects the 'All responses' sheet to be present.
# If not, you would first need to run the code that does:
# file_path = "250331 Leng Survey. Full download. Editable.xlsx"
# all_sheets_dict = pd.read_excel(file_path, sheet_name=None)
# --- END ASSUMPTION ---

sheet_name = 'All responses'
print(f"--- Analyzing columns of sheet: '{sheet_name}' ---")

try:
    # --- Get the target DataFrame ---
    if 'all_sheets_dict' not in locals():
        raise NameError("The dictionary 'all_sheets_dict' does not exist. Please run the reading code first.")
    if sheet_name not in all_sheets_dict:
        raise KeyError(f"Sheet name '{sheet_name}' not found in the dictionary. Available sheets: {list(all_sheets_dict.keys())}")

    # Assign the specific DataFrame to the variable name you mentioned
    df_all_responses = all_sheets_dict[sheet_name]

    # --- Create Metadata DataFrame (describing columns of df_all_responses) ---
    print("Calculating unique values for each column in df_all_responses...")
    column_unique_counts = df_all_responses.nunique()

    # Create a DataFrame to hold metadata about the columns
    # The index of this DataFrame will be the column names from df_all_responses
    column_metadata_df = pd.DataFrame({
        'UniqueValues': column_unique_counts,
        'DataType': df_all_responses.dtypes # Added data type back for context
    })

    # --- Add the ResponseType classification to the metadata DataFrame ---
    print("Classifying columns based on unique value count...")
    column_metadata_df['ResponseType'] = np.where(
        column_metadata_df['UniqueValues'] <= 10, # Condition
        'Fixed Response',                         # Value if True
        'Open Text'                               # Value if False
    )

    # --- Display the Column Metadata ---
    print("\n--- Column Metadata DataFrame ---")
    print("(Describes the columns in 'df_all_responses')")
    with pd.option_context('display.max_rows', None):
        print(column_metadata_df)

    # --- How to USE this metadata with df_all_responses ---
    print("\n--- Example Usage ---")

    # 1. Get a list of column names classified as 'Open Text'
    open_text_column_names = column_metadata_df[column_metadata_df['ResponseType'] == 'Open Text'].index.tolist()
    print(f"\nNumber of 'Open Text' columns found: {len(open_text_column_names)}")
    if open_text_column_names:
        print(f"First few 'Open Text' columns: {open_text_column_names[:5]}...") # Show first 5

    # 2. Get a list of column names classified as 'Fixed Response'
    fixed_response_column_names = column_metadata_df[column_metadata_df['ResponseType'] == 'Fixed Response'].index.tolist()
    print(f"\nNumber of 'Fixed Response' columns found: {len(fixed_response_column_names)}")
    if fixed_response_column_names:
        print(f"First few 'Fixed Response' columns: {fixed_response_column_names[:5]}...") # Show first 5

    # 3. Create a new DataFrame containing only the 'Open Text' columns from the original data
    if open_text_column_names:
        df_open_text_only = df_all_responses[open_text_column_names]
        print("\nCreated 'df_open_text_only' DataFrame with shape:", df_open_text_only.shape)
        # print(df_open_text_only.head()) # Uncomment to see the head

    # 4. Create a new DataFrame containing only the 'Fixed Response' columns
    if fixed_response_column_names:
        df_fixed_response_only = df_all_responses[fixed_response_column_names]
        print("Created 'df_fixed_response_only' DataFrame with shape:", df_fixed_response_only.shape)
        # print(df_fixed_response_only.head()) # Uncomment to see the head


except (NameError, KeyError) as e:
     print(f"Error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

print(f"\n--- Column analysis and classification complete for '{sheet_name}' ---")

--- Analyzing columns of sheet: 'All responses' ---
Calculating unique values for each column in df_all_responses...
Classifying columns based on unique value count...

--- Column Metadata DataFrame ---
(Describes the columns in 'df_all_responses')
                                                    UniqueValues  \
Which survey would you like to complete?                       2   
Do you work (currently or previously) as one of...           384   
Do you work (currently or previously) as one of...            74   
Which organisation do you work for?                          691   
In which healthcare setting do you work?                      28   
In which healthcare setting do you work?.1                    12   
In which region is the service where you most r...            20   
When did you last work in a healthcare setting ...             4   
In which organisation did you most recently wor...          2512   
In which healthcare setting did you most recent...            79   
In 

# Detecting suspicious responses

## 1. Completion time analysis
*Respondents answering unreasonably fast are suspicious (bots or not reading)*

In [4]:
import pandas as pd
import numpy as np

# --- ASSUMPTION ---
# This code assumes the 'df_all_responses' DataFrame already exists
# in your environment from the previous steps.
# If not, you would first need to run the code that loads it, e.g.:
# file_path = "250331 Leng Survey. Full download. Editable.xlsx"
# all_sheets_dict = pd.read_excel(file_path, sheet_name=None)
# df_all_responses = all_sheets_dict['All responses']
# --- END ASSUMPTION ---

print("\n--- Running Idea 1: Completion Time Analysis ---")

# Define the columns needed
start_col = 'Started'
end_col = 'Completed'

try:
    # --- Basic Checks ---
    if 'df_all_responses' not in locals():
         raise NameError("The DataFrame 'df_all_responses' does not exist. Please load it first.")
    if not all(col in df_all_responses.columns for col in [start_col, end_col]):
        missing = [col for col in [start_col, end_col] if col not in df_all_responses.columns]
        raise KeyError(f"Required columns missing from df_all_responses: {missing}")

    # --- 1. Convert to Datetime ---
    # Create new columns to avoid modifying originals immediately if preferred
    # errors='coerce' will turn unparseable dates into NaT (Not a Time)
    df_all_responses[f'{start_col}_dt'] = pd.to_datetime(df_all_responses[start_col], errors='coerce')
    df_all_responses[f'{end_col}_dt'] = pd.to_datetime(df_all_responses[end_col], errors='coerce')

    # --- 2. Calculate Duration ---
    df_all_responses['Duration'] = df_all_responses[f'{end_col}_dt'] - df_all_responses[f'{start_col}_dt']
    df_all_responses['Duration_sec'] = df_all_responses['Duration'].dt.total_seconds()

    # --- 3. Report on Missing Durations ---
    initial_rows = len(df_all_responses)
    valid_duration_count = df_all_responses['Duration_sec'].notna().sum()
    if initial_rows != valid_duration_count:
        print(f"\nWarning: Could not calculate duration for {initial_rows - valid_duration_count} out of {initial_rows} rows.")
        print("         This is likely due to missing or invalid data in 'Started' or 'Completed' columns.")

    # --- 4. Show Duration Statistics ---
    print("\n--- Distribution of Completion Times (in seconds) ---")
    if valid_duration_count > 0:
        # Use dropna() to calculate stats only on valid durations
        duration_stats = df_all_responses['Duration_sec'].dropna().describe(
            percentiles=[.01, .05, .10, .25, .5, .75, .90, .95, .99] # Include useful percentiles
        )
        # Make sure output format handles potentially large numbers
        with pd.option_context('display.float_format', '{:,.2f}'.format):
            print(duration_stats)
    else:
        print("No valid durations found to calculate statistics.")

    # --- 5. Define and Apply Threshold ---
    # Using 5th percentile as an example threshold for "too fast"
    percentile_to_flag = 0.05
    time_threshold_sec = None # Initialize

    if valid_duration_count > 0:
        time_threshold_sec = df_all_responses['Duration_sec'].dropna().quantile(percentile_to_flag)

    print(f"\n--- Flagging Responses Below {percentile_to_flag*100:.0f}th Percentile ---")
    if pd.notna(time_threshold_sec):
         print(f"Calculated Threshold: {time_threshold_sec:.2f} seconds")
         # Apply flag: True if Duration_sec is less than threshold
         df_all_responses['Flag_ShortTime'] = df_all_responses['Duration_sec'] < time_threshold_sec
         # Important: Ensure rows where duration couldn't be calculated (NaN) are NOT flagged
         df_all_responses['Flag_ShortTime'] = df_all_responses['Flag_ShortTime'].fillna(False)
    else:
         print("Could not calculate a valid threshold (perhaps too few valid durations). Setting all flags to False.")
         df_all_responses['Flag_ShortTime'] = False # Assign False to all rows

    # --- 6. Show Results ---
    flagged_count = df_all_responses['Flag_ShortTime'].sum()
    print(f"\n--- Results of Short Time Check ---")
    print(f"Total responses analyzed: {initial_rows}")
    print(f"Number of responses flagged as potentially too short (< {time_threshold_sec:.2f} sec): {flagged_count}")
    if initial_rows > 0:
        print(f"Percentage flagged: {flagged_count / initial_rows * 100:.2f}%")

    if flagged_count > 0:
        print("\n--- Sample of Flagged Responses (Sorted by Duration) ---")
        # Select relevant columns to display context
        cols_to_show = [
            'Duration_sec',
            'Flag_ShortTime',
            start_col, # Original timestamp cols
            end_col,
            # Add other potentially useful columns if they exist
            'Response ID',
            'Status',
            'Test response'
         ]
        # Filter out columns that might not exist in the specific df
        cols_to_show = [col for col in cols_to_show if col in df_all_responses.columns]

        # Show the top N fastest flagged responses
        with pd.option_context('display.max_rows', 20, 'display.max_columns', None, 'display.width', 1000):
             print(df_all_responses[df_all_responses['Flag_ShortTime']][cols_to_show].sort_values('Duration_sec').head(15)) # Show up to 15 fastest

except NameError as e:
     print(f"Error: {e}")
except KeyError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

print(f"\n--- Completion Time Analysis Finished ---")
print(f"The DataFrame 'df_all_responses' now has a 'Flag_ShortTime' column.")


--- Running Idea 1: Completion Time Analysis ---

--- Distribution of Completion Times (in seconds) ---
count       8,558.00
mean       13,604.58
std       112,978.50
min           120.00
1%            240.00
5%            300.00
10%           360.00
25%           540.00
50%           900.00
75%         1,680.00
90%         3,540.00
95%         8,049.00
99%       287,357.40
max     1,819,440.00
Name: Duration_sec, dtype: float64

--- Flagging Responses Below 5th Percentile ---
Calculated Threshold: 300.00 seconds

--- Results of Short Time Check ---
Total responses analyzed: 8558
Number of responses flagged as potentially too short (< 300.00 sec): 165
Percentage flagged: 1.93%

--- Sample of Flagged Responses (Sorted by Duration) ---
      Duration_sec  Flag_ShortTime             Started           Completed                           Response ID     Status  Test response
1122         120.0            True 2025-03-28 19:06:00 2025-03-28 19:08:00  8550336f-00e8-41ba-a645-770e10f9518c  co